In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.sparse import csc_matrix
from scipy.sparse import linalg

plt.style.use("default")
plt.rc("text", usetex=True)
plt.rc("font", family="cm")
plt.rcParams["grid.color"] = (0.5, 0.5, 0.5, 0.2)

In [ ]:
L = 10 
hbar = 1
m = 1
N_x = 100
N_y = 100
X, Y = np.meshgrid(np.linspace(0, L, N_x, dtype = float), 
                   np.linspace(0, L, N_y, dtype = float))
h_x = L/N_x
h_y = L/N_y

**Infinite Square Well**

In [ ]:
def get_infinite_well():
    V = np.zeros((N_x, N_y))
    return V

**Infinite Square Well with Step**

In [ ]:
def get_infinite_well_step(V_0=0, a=0, b=L):
    """
    V_0 = height of the step
    a = beginning of the step
    b = end of the step
    """
    V = np.zeros((N_x, N_y))
    for i, x in enumerate(np.linspace(0, L, N_x)):
        for j, y in enumerate(np.linspace(0, L, N_y)):
            if (x >= a and x <= b) and (y >= a and y <= b):
                V[i, j] = V_0
    return V            

**Harmonic Potential**

In [ ]:
def get_harmonic(omega=0):
    """
    omega = pulsation
    """
    V = np.zeros((N_x, N_y))
    for i, x in enumerate(np.linspace(0, L, N_x)):
        for j, y in enumerate(np.linspace(0, L, N_y)):
            V[i, j] = 0.5*omega*((x - 0.5*L)**2 + (y - 0.5*L)**2)
    return V

**Stationary Schrodinger Equation:** <br><br>
$-\frac{\hbar^2}{2m} \nabla^2 \psi(x, y) + V(x, y)\psi(x, y) = E\psi(x, y)$ <br><br>
$ \nabla^2 \psi(x, y) - 2V(x, y)\psi(x, y) = -2E\psi(x, y)$ <br><br>
let's define $M\psi(x, y) = \nabla^2 \psi(x, y) - 2V(x, y)\psi(x, y)$ and $H = -\frac{1}{2}M$ (finite differences + boundary conditions)

In [ ]:
def get_hamiltonian(V):
    """
    V = potential defined by previous functions
    for i in [2, N_x-1] and j in [2, N_y-1] --> the wave function is zero at the boundary
    """
    H = np.zeros(((N_x-2)*(N_y-2), (N_x-2)*(N_y-2)))
    for i1 in range(2, N_x):
        for j1 in range(2, N_y):
            l = (j1-2)*(N_x-2) + (i1-2) # variable in [0, (N_x-2)(N_y-2)-1]
            for i2 in range(2, N_x):
                for j2 in range(2, N_y):
                    m = (j2-2)*(N_x-2) + (i2-2) # variable in [0, (N_x-2)(N_y-2)-1]
                    if (i1+1 == i2) and (j1 == j2) and (i1+1 != N_x):
                        H[l, m] = 1/h_x**2
                    elif (i1-1 == i2) and (j1 == j2) and (i1-1 != 1):
                        H[l, m] = 1/h_x**2
                    elif (i1 == i2) and (j1+1 == j2) and (j1+1 != N_y):
                        H[l, m] = 1/h_y**2
                    elif (i1 == i2) and (j1-1 == j2) and (j1-1 != 1):
                        H[l, m] = 1/h_y**2
                    elif (i1 == i2) and (j1 == j2):
                        H[l, m] = -2*(1/h_x**2 + 1/h_y**2) -2*V[1:-1, 1:-1].reshape((N_x-2)*(N_y-2))[l]
    return H

In [ ]:
def solve_schrodinger(V, k):
    """
    V = potential defined by previous functions
    k = number of eigenvalues/eigenvectors to find
    """
    H = get_hamiltonian(V)
    sH = csc_matrix(H) # cast to scipy sparse matrix for better efficiency
    eigenvalues, eigenvectors = linalg.eigsh(sH, k=k)
    return eigenvalues, eigenvectors.T.reshape((k, N_x-2, N_y-2))

In [ ]:
def plot_solution_2D(eigenvector, fname):
    """
    eigenvector = (N_x-2)(N_y-2) solution returned by solve_schrodinger
    fname = filename
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5.5,5))
    contour = ax.contourf(X[1:-1, 1:-1], Y[1:-1, 1:-1], eigenvector**2, cmap=cm.viridis)
    ax.set_title('$|\psi(x,y)|^2$')
    ax.set_xlabel('x')
    ax.set_ylabel('y', rotation=0)
    fig.savefig(f"images/solution_2D_{fname}", dpi=300, bbox_inches="tight", pad_inches=0.2)


def plot_solution_3D(eigenvector, fname):
    """
    eigenvector = (N_x-2)(N_y-2) solution returned by solve_schrodinger
    fname = filename
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7,5), subplot_kw={"projection": "3d"})
    ax.plot_surface(X[1:-1, 1:-1], Y[1:-1, 1:-1], eigenvector**2, rstride=1, cstride=1, cmap=cm.coolwarm, linewidth=0, antialiased=False)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('$|\psi(x,y)|^2$', rotation=90)
    ax.tick_params(axis="both", which="major", labelsize=6)
    ax.tick_params(axis="both", which='minor', labelsize=6)
    ax.view_init(elev=12, azim=30)
    ax.zaxis.set_rotate_label(False)
    fig.savefig(f"images/solution_3D_{fname}", dpi=300, bbox_inches="tight", pad_inches=0.3)

In [ ]:
eigenvalues, eigenvectors = solve_schrodinger(get_infinite_well(), 10)

In [ ]:
%%capture
plot_solution_2D(eigenvectors[0], "well_0")
plot_solution_3D(eigenvectors[0], "well_0")

plot_solution_2D(eigenvectors[3], "well_3")
plot_solution_3D(eigenvectors[3], "well_3")

plot_solution_2D(eigenvectors[6], "well_6")
plot_solution_3D(eigenvectors[6], "well_6")